<a href="https://colab.research.google.com/github/megmarv/PsychoAI-/blob/main/ViTModel_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:








# Convert grayscale to RGB (ViT expects 3-channel input)
X_train = np.repeat(X_train, 3, axis=-1)
X_test = np.repeat(X_test, 3, axis=-1)

# Normalize pixel values (0-255 → 0-1)
X_train = X_train / 255.0
X_test = X_test / 255.0

# Convert labels to categorical (one-hot encoding)
num_classes = 7  # FER2013 has 7 emotion classes
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Define ViT model
def create_vit_model(input_shape=(48, 48, 3), num_classes=7):
    vit_model = keras.applications.VisionTransformer(
        image_size=input_shape[:2],  # Image dimensions
        patch_size=16,  # ViT processes 16x16 patches
        num_layers=12,  # Standard ViT depth
        num_heads=8,  # Multi-head attention
        hidden_dim=768,  # Embedding dimension
        mlp_dim=3072,  # Feedforward layer size
        dropout=0.1,
        classification=True,
        include_top=False,  # We add a custom classifier
        weights="imagenet"  # Load pretrained weights
    )

    # Add classifier head
    x = GlobalAveragePooling1D()(vit_model.output)
    x = Dense(256, activation="relu")(x)
    x = Dropout(0.5)(x)
    x = Dense(128, activation="relu")(x)
    x = Dropout(0.3)(x)
    output_layer = Dense(num_classes, activation="softmax")(x)  # Emotion classes

    # Define full model
    model = Model(inputs=vit_model.input, outputs=output_layer)

    return model

# Create model
vit_model = create_vit_model()

# Compile model
vit_model.compile(optimizer=Adam(learning_rate=3e-4),
                  loss="categorical_crossentropy",
                  metrics=["accuracy"])

# Train model
history = vit_model.fit(X_train, y_train, validation_data=(X_test, y_test),
                        epochs=15, batch_size=64)

# Evaluate model
test_loss, test_acc = vit_model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc * 100:.2f}%")


Hello World


In [4]:
# Install necessary libraries (if not installed)
!pip install tensorflow tensorflow-addons einops -q

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import efficientnet_v2
from einops.layers.tensorflow import Rearrange

In [5]:
import pandas as pd
import numpy as np

In [6]:
from google.colab import files
uploaded = files.upload()

Saving FER2013.zip to FER2013 (1).zip


In [7]:
import zipfile
import os

# Extract the uploaded zip file
zip_file = 'FER2013.zip'  # Update with your file name if different
extract_path = '/content/FER2013/'  # Path where the files will be extracted

with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Verify the extraction
os.listdir(extract_path)

['train', 'test']

In [8]:
# List contents of train and test folders
print(os.listdir(extract_path + 'train'))
print(os.listdir(extract_path + 'test'))

['fear', 'neutral', 'surprise', 'sad', 'happy', 'disgust', 'angry']
['fear', 'neutral', 'surprise', 'sad', 'happy', 'disgust', 'angry']


In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create ImageDataGenerators for training and testing
train_datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.2)  # Validation split = 20%
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Load training data
train_generator = train_datagen.flow_from_directory(
    os.path.join(extract_path, 'train'),  # Path to the train folder
    target_size=(48, 48),  # Resize images to 48x48 as required for ViT
    batch_size=64,
    class_mode='categorical',  # Multi-class classification (emotion categories)
    subset='training'  # Use the training subset
)

# Load validation data
validation_generator = train_datagen.flow_from_directory(
    os.path.join(extract_path, 'train'),  # Path to the train folder
    target_size=(48, 48),
    batch_size=64,
    class_mode='categorical',  # Multi-class classification
    subset='validation'  # Use the validation subset
)

# Load test data
test_generator = test_datagen.flow_from_directory(
    os.path.join(extract_path, 'test'),  # Path to the test folder
    target_size=(48, 48),
    batch_size=64,
    class_mode='categorical'  # Multi-class classification
)

Found 22968 images belonging to 7 classes.
Found 5741 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [10]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Create ViT model (Vision Transformer)
def create_vit_model(input_shape=(48, 48, 3), num_classes=7):
    # Input layer
    inputs = layers.Input(shape=input_shape)

    # Patch extraction
    x = layers.Conv2D(32, (3, 3), strides=(2, 2), padding="same")(inputs)
    x = layers.LayerNormalization()(x)
    x = layers.ReLU()(x)

    # Transformer block
    transformer_layer = layers.MultiHeadAttention(num_heads=4, key_dim=32)
    x = transformer_layer(x, x)
    x = layers.GlobalAveragePooling2D()(x)

    # Fully connected layer
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.2)(x)

    # Output layer
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    # Create the model
    model = models.Model(inputs=inputs, outputs=outputs)

    return model

# Create the ViT model
model = create_vit_model()

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 48, 48, 3)      │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 24, 24, 32)     │            896 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization       │ (None, 24, 24, 32)     │             64 │ conv2d[0][0]           │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu (ReLU)              │ (None, 24, 24, 32)     │              0 │ layer_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention      │ (None, 24, 24, 32)     │         16,800 │ re_lu[0][0],           │
│ (MultiHeadAttention)      │                        │                │ re_lu[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling2d  │ (None, 32)             │              0 │ multi_head_attention[… │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 128)            │          4,224 │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 128)            │              0 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 7)              │            903 │ dropout_1[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 22,887 (89.40 KB)

 Trainable params: 22,887 (89.40 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    verbose=1
)

Epoch 1/10
 44/359 ━━━━━━━━━━━━━━━━━━━━ 19:22 4s/step - accuracy: 0.2279 - loss: 1.8509

In [ ]:
# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

In [ ]:
# Step 1: Get predictions from the model
test_predictions = model.predict(test_generator, verbose=1)
test_labels = test_generator.classes
predicted_labels = np.argmax(test_predictions, axis=1)

# Step 2: Import necessary libraries
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt

# Step 3: Generate confusion matrix
cm = confusion_matrix(test_labels, predicted_labels)

# Step 4: Calculate overall accuracy
accuracy = accuracy_score(test_labels, predicted_labels)

# Step 5: Generate classification report with F1 scores
report = classification_report(test_labels, predicted_labels, target_names=test_generator.class_indices.keys())
print("\nClassification Report:\n", report)

# Step 6: Generate confusion matrix plot
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', xticklabels=test_generator.class_indices.keys(), yticklabels=test_generator.class_indices.keys())
plt.title("Confusion Matrix")
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

# Step 7: Display evaluation report
evaluation_report = {
    "Accuracy": accuracy,
    "Confusion Matrix": cm,
    "Classification Report": report
}

# Display evaluation summary
print("\n--- Evaluation Summary ---")
print(f"Accuracy: {accuracy:.4f}")
print("\nConfusion Matrix:\n", cm)
print("\n--- Classification Report ---\n", report)